### Linear regression from scratch

In this seminar you will implement a linear regression and train it using just the (stochastic) gradient descent, numpy and your brain.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

**Acknowledgements**: I've primariliy used the material from [Andrew Ng's Coursera course][1] for this, but have also been helped by [this article][2] and [this one][3]. I used some code for the animation from [this kernel][4].

  [1]: https://www.coursera.org/learn/machine-learning
  [2]: http://tillbergmann.com/blog/python-gradient-descent.html
  [3]: http://aimotion.blogspot.co.uk/2011/10/machine-learning-with-python-linear.html
  [4]: https://www.kaggle.com/ronaldtroncoso20/d/START-UMD/gtd/global-terrorism-trends-animation

Load the libraries and data...

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import http
import io
import requests

url = 'https://raw.githubusercontent.com/HSE-LaMBDA/MLatURL2019/master/day2/house-train.csv'
try:
    data = pd.read_csv(url)
except http.client.IncompleteRead as e:
    r = requests.get(url, timeout=10)
    b = io.StringIO(r.text)
    data = pd.read_csv(b)

Inspect the data

In [ ]:
data

### Univariate linear regression using an analytic solution

In this section, you will program the univariate least squares linear regression from scratch using an analytic formulae. 

Recall, that we're working with a problem of finding a weight $w$ such that
$$
\sum_i (y_i - w \cdot x_i)^2 \to \min,
$$
with $x_i, y_i \in R^1$, i.e. they are numbers. 

Recall that our least squares solution is of the form: 
$$
\widehat{w}_1 = 
    \frac{\sum_{i=1}^{\ell} (x_i - \mu_x) (y_i - \mu_y)}
        {\sum_{i=1}^{\ell} (x_i - \mu_x)^2}, \\
\widehat{w}_0 = \mu_y - \widehat{w}_1 \mu_x
$$
with $\mu_x = \frac{1}{\ell} \sum_{i=1}^{\ell} x_i,
\quad
\mu_y = \frac{1}{\ell} \sum_{i=1}^{\ell} y_i$

Hints: use `np.mean` to compute the mean. 

In [ ]:
def compute_weights_univariate(X, y):
    """
    Given feature array X [n_samples], target vector y [n_samples],
    compute the optimal least squares solution using the formulae above.
    """
    return <your code here>

In [ ]:
X = data['GrLivArea']
y = data['SalePrice']

w = compute_weights_univariate(X, y)
w

**How precise are we?**

We commonly evaluate predictions using MSE errors. These are defined as:
$$
MSE(y, \widehat{y}) = \frac 1 \ell \sum_i (y_i - w^T x_i)^2
$$
as implemented either by `sklearn.metrics.mean_squared_error` or simply by `np.mean((y - y_pred) ** 2)`.

We then use $\sqrt{MSE(y, \widehat{y})}$ to obtain numbers in the same dimensionality (units, not units squared).

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
w_0, w_1 = w
np.sqrt(
    mean_squared_error(y, np.dot(X, w_1) + w_0)
)

In [ ]:
f, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))
axs[0].scatter(y, np.dot(X, w_1) + w_0, alpha=.3)
axs[0].set_title('$y$ vs. $\widehat{y} = X \widehat{w}$')
axs[1].plot(y - (np.dot(X, w_1) + w_0))
axs[1].set_title('Error $y - X \widehat{w}$')

### Multivariate linear regression using an analytic solution

In this section, you will program the **multi**variate least squares linear regression from scratch using an analytic formulae. 

Recall, that we're working with a problem of finding a weight $w$ such that
$$
||\mathbf{y} - \mathbf{X} \mathbf{w}|| \to \min_{\mathbf{w}},
$$
with $\mathbf{x}_i \in R^n$, i.e. features are no longer supposed to be single numbers, but vectors of numbers (but still $y_i \in R^1$).

Recall that our least squares solution is of the form: 
$$
\mathbf{w} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}.
$$

We could account for the non-zero mean case ($\mathrm{E} \mathbf{y} \neq 0$) by either adding and subtracting the mean, or by using an additional feature in $\mathbf{X}$ set to all ones.

Hint: matrix inverse is computed by `np.linalg.inv`, and matrix-vector product by `np.dot`.

In [ ]:
def prepare_features(X):
    """
    Given feature array X [n_samples, n_features], preprocess it by
    standardizing and appending a column of 1s.
    """
    # First, standardize the input by subtracting mean and dividing by std
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    
    # Append a free term (a column of 1s)
    X = np.c_[np.ones(X.shape[0]), X]
    return X

In [ ]:
def compute_weights_multivariate(X, y):
    """
    Given feature array X [n_samples, 1], target vector y [n_samples],
    compute the optimal least squares solution using the formulae above.
    Don't forget the bias term!
    """
    return <your code here>

In [ ]:
X = data['GrLivArea']
X = np.atleast_2d(X).T
y = data['SalePrice']

w = compute_weights_multivariate(X, y)
w

In [ ]:
# note that we must run prepare_features(X) here, too, to obtain the exact same
# feature representation of the input
np.sqrt(
    mean_squared_error(y, np.dot(prepare_features(X), w))
)

Now that you have implemented the weight computation procedure, you can test it using our dataset, using 
* different features (or even their subsets!),
* different sub-samples of $x_i$s and $y_i$s.

In [ ]:
columns = [
    'LotFrontage',
    'LotArea',
    'MoSold',
    'YrSold',
    'OverallQual',
    'OverallCond',
    'YearBuilt',
    'YearRemodAdd',
    'MasVnrArea',
    'BsmtFinSF1',
    'GrLivArea'
]

In [ ]:
for col in columns: 
    X = data[col]
    X = np.atleast_2d(X).T
    y = data['SalePrice']
    
    # remove non-filled rows
    isnan_mask = np.any(np.isnan(X), axis=1) | np.isnan(y)
    X = X[~isnan_mask]
    y = y[~isnan_mask]
    
    w = compute_weights_multivariate(X, y)
    
    # note that we must construct the same feature vector to compute predictions
    # i.e. standardize and append column of 1s
    X = prepare_features(X)
    print('RMS error when using a single column "{col}" as regressor is {error:.2f}'
        .format(
            col=col,
            error=np.sqrt(mean_squared_error(y, np.dot(X, w)))
        )
    )

We could now easily combine multiple features for regression

In [ ]:
from itertools import combinations

In [ ]:
for cols in combinations(columns, 2): 
    X = data[list(cols)]
#     X = np.atleast_2d(X).T
    y = data['SalePrice']
    
    # remove non-filled rows
    isnan_mask = np.any(np.isnan(X), axis=1) | np.isnan(y)
    X = X[~isnan_mask]
    y = y[~isnan_mask]
    
    w = compute_weights_multivariate(X, y)
    
    # note that we must construct the same feature vector to compute predictions
    # i.e. standardize and append column of 1s
    X = prepare_features(X)
    print('RMS error when using a columns "{col}" as regressor is {error:.2f}'
        .format(
            col=cols,
            error=np.sqrt(mean_squared_error(y, np.dot(X, w)))
        )
    )

### Multivariante linear regression using the Gradient Descent

To use the gradient descent, we must 
* initialize the weights $\mathbf{w}$ somehow,
* find a way of computing the __gradient__ of our quality measure $L(\mathbf{y}, \widehat{\mathbf{y}})$ w.r.t. $\mathbf{w}$,
* starting from the initialization, iteratively update weights using the gradient descent: 
$$
\mathbf{w}^{(i+1)} \leftarrow \mathbf{w}^{(i)} - \gamma \nabla_{\mathbf{w}} L
$$

Since we choose $L(\mathbf{y}, \widehat{\mathbf{y}}) \equiv \frac 1 \ell ||\mathbf{y} - \mathbf{X} \mathbf{w} ||^2$, our gradient is $ \frac 2 \ell (\mathbf{y} - \mathbf{X} \mathbf{w}) \mathbf{X} $

In [ ]:
def compute_gradient(X, y, w):
    """
    Computes the gradient of MSE loss 
    for multivariate linear regression of X onto y 
    w.r.t. w, evaluated at the current w.
    """
    return <your code here>

In [ ]:
def gradient_descent(X, y, w_init, iterations=1, gamma=0.01):
    """
    Performs the required number of iterations of gradient descent.
    Parameters:
        X [n_objects, n_features]: matrix of featues
        y [n_objects]: responce (dependent) variable
        w_init: the value of w used as an initializer
        iterations: number of steps for gradient descent to compute
        gamma: learning rate (gradient multiplier)
    """
    costs, ws = [], []
    w = w_init
    for i in range(iterations):
        # Compute our cost in current point (before the gradient step)
        costs.append(mean_squared_error(y, np.dot(X, w)) / len(y))
        
        # Remember our weights w in current point
        ws.append(w)
        
        # Compute gradient for w
        w_grad = <your code here>
        
        # Update the current weight w using the formula above (see comments)
        w = <your code here>
        
    return costs, ws

How run the gradient descent to find the optimal weights vector.

In [ ]:
X = data['GrLivArea']
X = np.atleast_2d(X).T
X = prepare_features(X)
y = data['SalePrice']

gamma = 0.01  # Step size
iterations = 200  # No. of iterations
np.random.seed(123)  # Set the seed
w_init = np.random.rand(X.shape[1]) #Pick some random values to start with

# Pass the relevant variables to the function and get the new values back...
costs, ws = gradient_descent(X, y, w_init, iterations, gamma)
w = ws[-1]

# Print the results...
print("Gradient Descent: {:.2f}, {:.2f}".format(w[0], w[1]))

The following 3 code cells display the cost function and its progress as the learning continues.

In [ ]:
#Plot the cost function...
plt.title('Cost Function J')
plt.xlabel('No. of iterations')
plt.ylabel('Cost')
plt.plot(costs)
plt.show()

In [ ]:
#Animation
def animate_to_gif(X, y, costs, out_filename='animation.gif'):

    fig = plt.figure()
    ax = plt.axes()
    plt.title('Sale Price vs Living Area')
    plt.xlabel('Living Area in square feet (normalised)')
    plt.ylabel('Sale Price ($)')
    plt.scatter(X[:,1], y, color='red')
    line, = ax.plot([], [], lw=2)
    annotation = ax.text(-1, 700000, '')
    annotation.set_animated(True)
    plt.close()

    #Generate the animation data,
    def init():
        line.set_data([], [])
        annotation.set_text('')
        return line, annotation

    # animation function.  This is called sequentially
    def animate(i):
        x = np.linspace(-5, 20, 1000)
        y = ws[i][1]*x + ws[i][0]
        line.set_data(x, y)
        annotation.set_text('Cost = %.2f e10' % (past_costs[i]/10000000000))
        return line, annotation

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=iterations, interval=0, blit=True)

    anim.save(out_filename, writer='pillow', fps = 30)
    

def animation_display(filename='animation.gif'):
    #Display the animation...
    import io
    import base64
    from IPython.display import HTML

    video = io.open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

In [ ]:
out_filename = 'animation_gd.gif'

animate_to_gif(X, y, costs, out_filename)

#Display the animation...
import io
import base64
from IPython.display import HTML

video = io.open(out_filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

### Multivariante linear regression using the Stochastic Gradient Descent

In Stochastic Gradient Descent (SGD), we must do the same as for vanilla GD
* initialize the weights $\mathbf{w}$ somehow,
* find a way of computing the __gradient__ of our quality measure $L(\mathbf{y}, \widehat{\mathbf{y}})$ w.r.t. $\mathbf{w}$,
* starting from the initialization, iteratively update weights using the gradient descent: 
$$
\mathbf{w}^{(i+1)} \leftarrow \mathbf{w}^{(i)} - \gamma \nabla_{\mathbf{w}} L
$$
**but** we now compute the gradient with respect to __a single sample__ $(\mathbf{x}_i, y_i)$ only.

Thus, since we choose $L(y_i, \widehat{y}_i) \equiv (y_i - \mathbf{x}_i^T \mathbf{w} )^2$, 
our gradient is $ 2 (y_i - \mathbf{x}_i^T \mathbf{w}) \mathbf{x}_i $.

In fact, we can still use the already implemented `compute_gradient` function, but now we should only pass inside the subset (one) input vector.

In [ ]:
def stochastic_gradient_descent(X, y, w_init, iterations=1, gamma=0.01):
    """
    Performs the required number of iterations of stochastic gradient descent.
    Parameters:
        X [n_objects, n_features]: matrix of featues
        y [n_objects]: responce (dependent) variable
        w_init: the value of w used as an initializer
        iterations: number of steps for gradient descent to compute
        gamma: learning rate (gradient multiplier)
    """
    costs, ws = [], []
    w = w_init
    for i in range(iterations):
        # Compute our cost in current point (before the gradient step)
        costs.append(mean_squared_error(y, np.dot(X, w)) / len(y))
        
        # Remember our weights w in current point
        ws.append(w)
        
        # Select a random sample from the dataset
        i = <your code here>
        
        # Compute gradient for w
        w_grad = <your code here>
        
        # Update the current weight w using the formula above (see comments)
        w = <your code here>
        
    return costs, ws

In [ ]:
X = data['GrLivArea']
X = np.atleast_2d(X).T
X = prepare_features(X)
y = data['SalePrice']

gamma = 0.01  # Step size
iterations = 200  # No. of iterations
np.random.seed(123)  # Set the seed
w_init = np.random.rand(X.shape[1]) #Pick some random values to start with

# Pass the relevant variables to the function and get the new values back...
costs, ws = stochastic_gradient_descent(X, y, w_init, iterations, gamma)
w = ws[-1]

# Print the results...
print("Gradient Descent: {:.2f}, {:.2f}".format(w[0], w[1]))

Note that the cost function is not as smooth anymore

In [ ]:
#Plot the cost function...
plt.title('Cost Function J')
plt.xlabel('No. of iterations')
plt.ylabel('Cost')
plt.plot(costs)
plt.show()

Also note how, along with training, the line estimate is more noisy.

In [ ]:
out_filename = 'animation_sgd.gif'

animate_to_gif(X, y, costs, out_filename)

#Display the animation...
import io
import base64
from IPython.display import HTML

video = io.open(out_filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

### Train / Validation split

We commonly set aside a portion of data for testing our model. In typical scenarios, this corresponds to n-fold cross-validation, but here for the sake of simplicity we only show the train/test split.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data[columns]
y = data['SalePrice']

# this is common as many columns tend to be empty for some reason
isnan_mask = np.any(np.isnan(X), axis=1) | np.isnan(y)
X = X[~isnan_mask].values
y = y[~isnan_mask].values
# X = prepare_features(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
w = compute_weights_multivariate(X_train, y_train)
w

In [ ]:
train_rmse = np.sqrt(mean_squared_error(y_train, np.dot(prepare_features(X_train), w)))
test_rmse = np.sqrt(mean_squared_error(y_test, np.dot(prepare_features(X_test), w)))

print('Train error: ', train_rmse)
print('Test error: ', test_rmse)

It turns out that for many cases, validation-set MSE will be very different from MSE computed on train test. Regularization could be seen as one of the ways of tuning (additionally penalizing) the model for being _too well_ fitted on training data. Thus, performance on the validation set is more consistent with that on the training set.

### Regularized multivariate regression

The regularized least squares are given by the formula:
$$
\mathbf{w}^* =
  ( \mathbf{X}^{\intercal} \mathbf{X} + \alpha \mathbf{I})^{-1}
  \mathbf{X}^{\intercal} \mathbf{y}
$$

In [ ]:
def compute_weights_multivariate_l2reg(X, y, reg=1e-3):
    """
    Given feature array X [n_samples, 1], target vector y [n_samples],
    compute the optimal least squares solution using the formulae above.
    Don't forget the bias term!
    """
    return <your code here>

In [ ]:
w_reg = compute_weights_multivariate_l2reg(X_train, y_train, reg=0)

train_rmse = np.sqrt(mean_squared_error(y_train, np.dot(prepare_features(X_train), w_reg)))
test_rmse = np.sqrt(mean_squared_error(y_test, np.dot(prepare_features(X_test), w_reg)))

print('Train error: ', train_rmse)
print('Test error: ', test_rmse)

In [ ]:
reg_values = []
train_rmses, test_rmses = [], []
for reg in np.linspace(0.1, 100, 100):
    w_reg = compute_weights_multivariate_l2reg(X_train, y_train, reg=reg)
    reg_values.append(reg)

    train_rmse = np.sqrt(mean_squared_error(y_train, np.dot(prepare_features(X_train), w_reg)))
    test_rmse = np.sqrt(mean_squared_error(y_test, np.dot(prepare_features(X_test), w_reg)))
    train_rmses.append(train_rmse)
    test_rmses.append(test_rmse)
    
plt.plot(reg_values, train_rmses, 'r.', label='Train set')
plt.plot(reg_values, test_rmses, 'k.', label='Validation set')
plt.legend()

Thus, if we choose $\alpha \approx 50$, we will obtain lower performance on training set, but a more consistent quality estimate for previously unseen instances.

One question that's remaining is -- __how to choose $\alpha$, in practice__?

### Multivariate regression with a prior on weights

Note that the expression for Bayesian regression is the same compared to L2 regression. Yet, it has an important distinction -- the weights prior: $\mathbf{w} \sim \mathcal{N}(0, \Sigma_w)$. 

Let $\Sigma_w = \sigma^2_w \mathbf{I}$. If we estimate $\sigma^2_w$ using the training set, we end up having a more principled way of estimating $\mathbf{w}$.

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
y = y.values

In [ ]:
kfold = KFold(n_splits=10)
ws = []
for train_idx, test_idx in kfold.split(X, y):
    X_train, y_train = X[train_idx], y[train_idx]
    w = compute_weights_multivariate(X_train, y_train)
    ws.append(w)

In [ ]:
np.mean(ws, axis=0), np.std(ws, axis=0)

In [ ]:
np.mean(np.std(ws, axis=0))